# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# OpenAI API key sanity check
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment

system_prompt = """
You are a tech expert and know every coding language, and can give 
nice, detailed and simple explanations for the given questions.
"""

In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
# Get gpt-4o-mini to answer, with streaming

def stream_brochure():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [7]:
stream_brochure()

Certainly! Let's break down the code snippet you provided:

python
yield from {book.get("author") for book in books if book.get("author")}

### Understanding Each Part of the Code

1. **`yield from`**:
   - This is a statement used in Python generators. It allows a generator to yield values from another iterable. When you use `yield from`, it essentially delegates the yielding of values to the iterable that follows it. 

2. **Set Comprehension: `{...}`**:
   - The part inside the curly braces `{...}` is a set comprehension. Set comprehensions are used to create a set in Python—an unordered collection of unique elements.
   - In this case, the set comprehension is populated with values derived from the expression `book.get("author")` for each `book` in the `books` collection.

3. **Iterating through `books`**:
   - `for book in books`: This part iterates through each `book` in a collection called `books`. It's assumed that `books` is a list (or another iterable) of dictionaries, where each dictionary represents a book.

4. **`book.get("author")`**:
   - This expression fetches the value associated with the key `"author"` from each `book` dictionary. The `get` method is used here because it safely returns `None` if the key does not exist, instead of raising a KeyError.

5. **`if book.get("author")`**:
   - This is a conditional filter. It checks if the value fetched by `book.get("author")` is truthy (non-null and non-empty). This means that only books with a valid author will be considered for inclusion in the set.

### What the Code Does
The entire line of code constructs a set of unique authors from a list of books, filtering out any books that do not have an author.

- **Unique Result**: Since it uses a set comprehension, any duplicate author names will only appear once in the final set due to the properties of a set (which only stores unique values).

- **Yields Authors**: After creating the set of authors, `yield from` will yield each author name one at a time when this generator function is executed.

### Example
Here's a simple example to illustrate:

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},
]

def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Calling the generator
for author in get_unique_authors(books):
    print(author)


**Output**:

Author 1
Author 2


### Conclusion
To summarize, the code snippet efficiently collects and yields unique authors from a list of book dictionaries, filtering out any entries without an author. This approach is useful for extracting specific data while ensuring that duplicates are eliminated.

In [12]:
# Get Llama 3.2 to answer

def llama_answer():
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [13]:
llama_answer()

**Code Breakdown**

This line of code uses a combination of dictionary methods (`get`), list comprehension, and `yield from` to achieve an interesting result.

Here's a step-by-step explanation:

1. `{book.get("author") for book in books if book.get("author")}` is a dictionary comprehension (sometimes called nested dictionary comprehension). It iterates over the keys in the `books` dictionary.
2. The syntax `for book in books if book.get("author")` filters the iteration to only include dictionaries (`book`) that have a "author" key present.

The resulting dictionary comprehension returns an iterable view object, which is an iterator over the values associated with the matching keys.

3. `yield from ...` is a built-in Python keyword used in conjunction with generators (functions defined with the `yield` keyword). When used, it de-legitimizes the expression to be iterated as is, yielding each item that was originally present.

**What happens when you combine these parts?**

When this code snippet is executed:

* It'll skip dictionaries (`book`) in the original `books` array where there's no "author" key. This effectively creates a view of only those books with authors.

* For each book dictionary (with an author), it yields that author's name as an element to create another iterable generator object, **only if** another book has the same author, **AND**, it is also accessible through `books`, or simply: if there exists at least one matching "author" in a book of our current iteration.

The code appears to achieve several things:

- Filter out empty lists (where `book` has no "author".
  
  e.g.,
   - If the list [1], then when processing "title":

      python
      {
        key: value 
        for key, value in books.items() if book.get("author")
      }


   becomes 
     **{['value']

python
{
key: value
for  (book):key in book.keys()
if book.find ("author") > -1}


- In case multiple items have the same "author".

 So, this expression essentially builds a nested generator where when yielded it produces values by iterating through all elements of `books` (`dict() or an array`).

In [ ]:
response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))